In [ ]:
!pip install torch==1.1.0 tensorly==0.4.0

In [ ]:
!pip install wandb

In [ ]:
!nvidia-smi

Sat Dec 26 12:02:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from time import time
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac, partial_tucker
import torch
import torch.nn as nn
import wandb

torch.manual_seed(42)

Using numpy backend.


In [ ]:
device = torch.device("cpu")

In [ ]:
tl.set_backend('pytorch')

Using pytorch backend.


In [ ]:
def cp_decomposition_conv_layer(layer, rank):
    """ Gets a conv layer and a target rank, 
        returns a nn.Sequential object with the decomposition """

    # Perform CP decomposition on the layer weight tensorly. 
    last, first, vertical, horizontal = \
        parafac(layer.weight.data, rank=rank, init='svd')

    pointwise_s_to_r_layer = torch.nn.Conv2d(in_channels=first.shape[0], \
            out_channels=first.shape[1], kernel_size=1, stride=1, padding=0, 
            dilation=layer.dilation, bias=False)

    depthwise_vertical_layer = torch.nn.Conv2d(in_channels=vertical.shape[1], 
            out_channels=vertical.shape[1], kernel_size=(vertical.shape[0], 1),
            stride=1, padding=(layer.padding[0], 0), dilation=layer.dilation,
            groups=vertical.shape[1], bias=False)

    depthwise_horizontal_layer = \
        torch.nn.Conv2d(in_channels=horizontal.shape[1], \
            out_channels=horizontal.shape[1], 
            kernel_size=(1, horizontal.shape[0]), stride=layer.stride,
            padding=(0, layer.padding[0]), 
            dilation=layer.dilation, groups=horizontal.shape[1], bias=False)

    pointwise_r_to_t_layer = torch.nn.Conv2d(in_channels=last.shape[1], \
            out_channels=last.shape[0], kernel_size=1, stride=1,
            padding=0, dilation=layer.dilation, bias=True)

    depthwise_horizontal_layer.weight.data = \
        torch.transpose(horizontal, 1, 0).unsqueeze(1).unsqueeze(1)
    depthwise_vertical_layer.weight.data = \
        torch.transpose(vertical, 1, 0).unsqueeze(1).unsqueeze(-1)
    pointwise_s_to_r_layer.weight.data = \
        torch.transpose(first, 1, 0).unsqueeze(-1).unsqueeze(-1)
    pointwise_r_to_t_layer.weight.data = last.unsqueeze(-1).unsqueeze(-1)

    new_layers = [pointwise_s_to_r_layer, depthwise_vertical_layer, \
                    depthwise_horizontal_layer, pointwise_r_to_t_layer]
    
    return nn.Sequential(*new_layers).to(device)

In [ ]:
def tucker_decomposition_conv_layer(layer,ranks):
    """ Gets a conv layer, 
        returns a nn.Sequential object with the Tucker decomposition.
        The ranks are estimated with a Python implementation of VBMF
        https://github.com/CasvandenBogaard/VBMF
    """
    core, [last, first] = \
        partial_tucker(layer.weight.data, \
            modes=[0, 1], ranks=ranks, init='svd')

    # A pointwise convolution that reduces the channels from S to R3
    first_layer = torch.nn.Conv2d(in_channels=first.shape[0], \
            out_channels=first.shape[1], kernel_size=1,
            stride=1, padding=0, dilation=layer.dilation, bias=False)

    # A regular 2D convolution layer with R3 input channels 
    # and R3 output channels
    core_layer = torch.nn.Conv2d(in_channels=core.shape[1], \
            out_channels=core.shape[0], kernel_size=layer.kernel_size,
            stride=layer.stride, padding=layer.padding, dilation=layer.dilation,
            bias=False)

    # A pointwise convolution that increases the channels from R4 to T
    last_layer = torch.nn.Conv2d(in_channels=last.shape[1], \
        out_channels=last.shape[0], kernel_size=1, stride=1,
        padding=0, dilation=layer.dilation, bias=True)


    first_layer.weight.data = \
        torch.transpose(first, 1, 0).unsqueeze(-1).unsqueeze(-1)
    last_layer.weight.data = last.unsqueeze(-1).unsqueeze(-1)
    core_layer.weight.data = core

    new_layers = [first_layer, core_layer, last_layer]
    return nn.Sequential(*new_layers).to(device)

In [ ]:
wandb.login()

wandb: Currently logged in as: omshri (use `wandb login --relogin` to force relogin)


True

In [ ]:
checks = [
  [8,1,10,10,1,3,3],
  [8,3,10,10,1,3,3],
  [8,1,10,10,4,3,3],
  [8,3,10,10,4,3,3],
  [8,1,10,10,2,3,3],
  [8,3,10,10,2,3,3],
  [8,1,10,10,8,3,3],
  [8,3,10,10,8,3,3],
  [8,3,10,10,1,5,5],
  [8,6,10,10,1,5,5],
  [8,3,10,10,4,5,5],
  [8,6,10,10,4,5,5],
  [8,3,20,20,1,5,5],
  [8,3,20,20,1,5,5],
  [8,1,20,20,4,5,5],
  [8,3,20,20,4,5,5],
  [8,1,20,20,1,8,8],
  [8,3,20,20,1,8,8],
  [8,1,20,20,1,11,11],
  [8,3,20,20,1,11,11],        
]
print(len(checks))

20


In [ ]:
for i in checks:

  N,C,H,W,M,R,S = i

  wandb.init(project="conv_time_cptk_gpu",name="N:"+str(N)+", C:"+str(C)+", H:"+str(H)+", W:"+str(W)+", M:"+str(M)+", R:"+str(R)+", S:"+str(S),reinit=True)

  conv = nn.Conv2d(C,M,(R,S),bias=False)

  inp = torch.rand(N,C,H,W).to(device)
  ker = conv.weight

  rank = max(ker.data.numpy().shape)//3
  
  cp = cp_decomposition_conv_layer(conv,rank)
  tk = tucker_decomposition_conv_layer(conv,[rank,rank])

  start = time()
  out = cp(inp)
  cp_time = time()-start

  start = time()
  out = tk(inp)
  tk_time = time()-start

  wandb.log({'cp':cp_time*1000})
  wandb.log({'tk':tk_time*1000})

  wandb.finish()

/usr/local/lib/python3.6/dist-packages/tensorly/backend/pytorch_backend.py:152: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  solution, _ = torch.gesv(matrix2, matrix1)


cp,1.72901
_step,1
_runtime,1
_timestamp,1608984165
tk,0.73695


cp,▁
_step,▁█
_runtime,▁▁
_timestamp,▁▁
tk,▁


cp,1.58477
_step,1
_runtime,1
_timestamp,1608984169
tk,1.07121


cp,▁
_step,▁█
_runtime,▁▁
_timestamp,▁▁
tk,▁


...

cp,▁
_step,▁█
_runtime,▁▁
_timestamp,▁▁
tk,▁


cp,1.97816
_step,1
_runtime,0
_timestamp,1608984239
tk,1.21784


cp,▁
_step,▁█
_runtime,▁▁
_timestamp,▁▁
tk,▁


cp,1.37806
_step,1
_runtime,1
_timestamp,1608984244
tk,0.97728


cp,▁
_step,▁█
_runtime,▁▁
_timestamp,▁▁
tk,▁
